In [1]:
# https://towardsdatascience.com/recommendation-systems-explained-a42fc60591ed
# https://www.geeksforgeeks.org/ml-feature-scaling-part-2/
# https://realpython.com/build-recommendation-engine-collaborative-filtering/
# https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DA DS/Modeled_mobile.csv')

In [4]:
# Make copy of data frame
copy_df = df.copy()

In [5]:
# Using OneHotEncoder convert availability column from categorical columns to numerical columns

onehot_encoder = OneHotEncoder()
availability = onehot_encoder.fit_transform(df[['availability']])

onehot_df = pd.DataFrame(availability.toarray(), columns=onehot_encoder.get_feature_names_out(['availability']))

copy_df = pd.concat([copy_df, onehot_df], axis=1)

copy_df.drop('availability', axis=1, inplace=True)

In [6]:
# With LabelEncoder convert categorical columns to numerical columns

categorical_col = ['Brand', 'PhoneModel', 'PhoneColor', 'Display', 'Camera', 'Battery', 'Processor']

label_encoder = LabelEncoder()
for column in categorical_col:
    copy_df[column] = label_encoder.fit_transform(copy_df[column])

In [7]:
# Feature Scaling

columns_to_scale = ['NetworkType', 'ProductRating', 'DiscountPrice', 'OriginalPrice', 'Discount', 'Num_Ratings', 'Num_Reviews', 'RAM', 'ROM', 'Expandable']
scaler = MinMaxScaler()
copy_df[columns_to_scale] = scaler.fit_transform(copy_df[columns_to_scale])

In [8]:
copy_df.shape

(8477, 28)

In [9]:
copy_df.describe(include='all')

,Brand,PhoneModel,PhoneColor,NetworkType,ProductRating,DiscountPrice,OriginalPrice,Discount,Num_Ratings,Num_Reviews,...,bank_offer,saver_deal,Brand_Warranty,year_month_Warranty,accessories_Warranty,rplc_rep_ser_Warranty,availability_Available,availability_Coming Soon,availability_Currently unavailable,availability_Pre Order
count,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,...,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000,8477.000000
mean,87.919665,1694.295152,603.556801,0.127639,0.739495,0.083297,0.095182,0.168952,0.020599,0.012506,...,0.866226,0.025481,0.744839,0.989383,0.545948,0.043412,0.360505,0.110180,0.528725,0.000590
std,47.281911,984.549227,311.144039,0.330082,0.232684,0.121521,0.136193,0.204931,0.066936,0.046027,...,0.340429,0.157589,0.435977,0.102496,0.497914,0.203794,0.480175,0.313133,0.499204,0.024281
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58.000000,810.000000,397.000000,0.000000,0.740000,0.008000,0.010521,0.000000,0.000063,0.000047,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,93.000000,1882.000000,720.000000,0.000000,0.820000,0.047363,0.055258,0.077922,0.000918,0.000558,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,116.000000,2540.000000,720.000000,0.000000,0.860000,0.094732,0.107895,0.298701,0.008423,0.005110,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,169.000000,3310.000000,1182.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
copy_df.columns

Index(['Brand', 'PhoneModel', 'PhoneColor', 'NetworkType', 'ProductRating',
       'DiscountPrice', 'OriginalPrice', 'Discount', 'Num_Ratings',
       'Num_Reviews', 'Display', 'Camera', 'Battery', 'Processor', 'RAM',
       'ROM', 'Expandable', 'exchange_offer', 'bank_offer', 'saver_deal',
       'Brand_Warranty', 'year_month_Warranty', 'accessories_Warranty',
       'rplc_rep_ser_Warranty', 'availability_Available',
       'availability_Coming Soon', 'availability_Currently unavailable',
       'availability_Pre Order'],
      dtype='object')

In [11]:
# Based on all Columns Recommend 10 Mobiles
n_neighbors = 10
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(copy_df)

item_index = 50 # Select Mobile Phone
query_item = copy_df.iloc[item_index].values.reshape(1, -1)

distances, indices = nbrs.kneighbors(query_item)

recommendations = df.iloc[indices[0]]

print("selected Mobile:")
selected_item = df.iloc[item_index]
print(selected_item)

print("\n Recommended Mobiles:")
recommendations

selected Mobile:
Brand                                           SAMSUNG
PhoneModel                         SAMSUNG GALAXY M33  
PhoneColor                              DEEP OCEAN BLUE
NetworkType                                           5
ProductRating                                       4.2
DiscountPrice                                     16300
OriginalPrice                                     24999
Discount                                             34
availability                                  Available
Num_Ratings                                       11095
Num_Reviews                                         813
Display                   '16.76 cm (6.6 inch) Display'
Camera                               '50MP Rear Camera'
Battery                              '6000 mAh Battery'
Processor                                  NA Processor
RAM                                                 6.0
ROM                                               128.0
Expandable                     

,Brand,PhoneModel,PhoneColor,NetworkType,ProductRating,DiscountPrice,OriginalPrice,Discount,availability,Num_Ratings,...,RAM,ROM,Expandable,exchange_offer,bank_offer,saver_deal,Brand_Warranty,year_month_Warranty,accessories_Warranty,rplc_rep_ser_Warranty
50,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16300,24999,34,Available,11095,...,6.0,128.0,0.0,0,1,0,1,1,0,0
30,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16385,24999,34,Available,11095,...,6.0,128.0,0.0,0,1,0,1,1,0,0
39,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,17379,25999,33,Available,3173,...,8.0,128.0,0.0,0,1,0,1,1,0,0
21,SAMSUNG,SAMSUNG GALAXY M33,EMARLD BROWN,5,4.2,16189,22999,29,Available,11095,...,6.0,128.0,0.0,0,1,0,1,1,0,0
336,SAMSUNG,SAMSUNG GALAXY M33,EMERALD BROWN,5,4.2,16990,24999,32,Currently unavailable,11095,...,6.0,128.0,0.0,0,1,0,1,1,0,0
328,SAMSUNG,SAMSUNG GALAXY M33,EMERALD BROWN,5,4.2,16990,24999,32,Currently unavailable,11095,...,6.0,128.0,0.0,0,1,0,1,1,0,0
346,SAMSUNG,SAMSUNG GALAXY M51,ELECTRIC BLUE,0,4.4,17996,28999,37,Currently unavailable,2021,...,6.0,128.0,0.0,0,1,0,1,1,0,0
64,SAMSUNG,SAMSUNG GALAXY A14,DARK RED,5,4.0,17999,20999,14,Available,2398,...,6.0,128.0,1024.0,1,1,0,1,1,1,0
84,SAMSUNG,SAMSUNG GALAXY A14,DARK RED,5,4.1,19999,22999,13,Available,1386,...,8.0,128.0,1024.0,1,1,0,1,1,1,0
88,SAMSUNG,SAMSUNG GALAXY A14,DARK RED,5,4.1,15999,18499,13,Available,1515,...,4.0,64.0,1024.0,1,1,0,1,1,1,0


In [12]:
# Based on the Popularity Recommend 10 Mobiles
df['PopularityScore'] = df['ProductRating'] * df['Num_Ratings'] * df['Num_Reviews']
popularity_features = ['ProductRating', 'Num_Ratings', 'Num_Reviews']

n_neighbors = 10
knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(copy_df[popularity_features])

select_item_index = 50 # Select Mobile Phone
test_item = copy_df.iloc[select_item_index][popularity_features].values.reshape(1, -1)

distances, indices = knn.kneighbors(test_item)

recommended_indices = indices.flatten()
recommended_phones = df.iloc[recommended_indices]

recommended_phones = recommended_phones.sort_values(by='PopularityScore', ascending=False)

print("selected Mobile:")
print(df.iloc[select_item_index])

print("\n Recommended Mobiles:")
recommended_phones

selected Mobile:
Brand                                           SAMSUNG
PhoneModel                         SAMSUNG GALAXY M33  
PhoneColor                              DEEP OCEAN BLUE
NetworkType                                           5
ProductRating                                       4.2
DiscountPrice                                     16300
OriginalPrice                                     24999
Discount                                             34
availability                                  Available
Num_Ratings                                       11095
Num_Reviews                                         813
Display                   '16.76 cm (6.6 inch) Display'
Camera                               '50MP Rear Camera'
Battery                              '6000 mAh Battery'
Processor                                  NA Processor
RAM                                                 6.0
ROM                                               128.0
Expandable                     

,Brand,PhoneModel,PhoneColor,NetworkType,ProductRating,DiscountPrice,OriginalPrice,Discount,availability,Num_Ratings,...,ROM,Expandable,exchange_offer,bank_offer,saver_deal,Brand_Warranty,year_month_Warranty,accessories_Warranty,rplc_rep_ser_Warranty,PopularityScore
506,SAMSUNG,SAMSUNG GALAXY J2 CORE,BLACK,0,4.2,6999,6999,0,Coming Soon,11047,...,16.0,256.0,0,0,0,1,1,1,0,40458532.8
582,SAMSUNG,SAMSUNG GALAXY J2 CORE,GOLD,0,4.2,4999,6400,21,Currently unavailable,11047,...,8.0,256.0,0,1,0,1,1,1,0,40458532.8
328,SAMSUNG,SAMSUNG GALAXY M33,EMERALD BROWN,5,4.2,16990,24999,32,Currently unavailable,11095,...,128.0,0.0,0,1,0,1,1,0,0,38537373.0
30,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16385,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
16,SAMSUNG,SAMSUNG GALAXY M33,MYSTIQUE GREEN,5,4.2,16490,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
66,SAMSUNG,SAMSUNG GALAXY M33,MYSTIQUE GREEN,5,4.2,15990,24999,36,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
336,SAMSUNG,SAMSUNG GALAXY M33,EMERALD BROWN,5,4.2,16990,24999,32,Currently unavailable,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
50,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16300,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
21,SAMSUNG,SAMSUNG GALAXY M33,EMARLD BROWN,5,4.2,16189,22999,29,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
60,SAMSUNG,SAMSUNG GALAXY M33,MYSTIQUE GREEN,5,4.2,16900,24999,32,Available,11095,...,128.0,0.0,0,1,1,1,1,0,0,37884987.0


In [13]:
# Based on the Price Recommend 10 Mobiles
knn_features = ['DiscountPrice', 'OriginalPrice',	'Discount']

n_neighbors = 10
knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(copy_df[knn_features])

select_item_index = 50 # Select Mobile Phone
test_item = copy_df.iloc[select_item_index][knn_features].values.reshape(1, -1)

distances, indices = knn.kneighbors(test_item)

recommended_indices = indices.flatten()
recommended_phones = df.iloc[recommended_indices]

recommended_phones = recommended_phones.sort_values(by='PopularityScore', ascending=False)

print("selected Mobile:")
print(df.iloc[select_item_index])

print("\n Recommended Mobiles:")
recommended_phones

selected Mobile:
Brand                                           SAMSUNG
PhoneModel                         SAMSUNG GALAXY M33  
PhoneColor                              DEEP OCEAN BLUE
NetworkType                                           5
ProductRating                                       4.2
DiscountPrice                                     16300
OriginalPrice                                     24999
Discount                                             34
availability                                  Available
Num_Ratings                                       11095
Num_Reviews                                         813
Display                   '16.76 cm (6.6 inch) Display'
Camera                               '50MP Rear Camera'
Battery                              '6000 mAh Battery'
Processor                                  NA Processor
RAM                                                 6.0
ROM                                               128.0
Expandable                     

,Brand,PhoneModel,PhoneColor,NetworkType,ProductRating,DiscountPrice,OriginalPrice,Discount,availability,Num_Ratings,...,ROM,Expandable,exchange_offer,bank_offer,saver_deal,Brand_Warranty,year_month_Warranty,accessories_Warranty,rplc_rep_ser_Warranty,PopularityScore
2754,MOTOROLA,MOTOROLA G62,FROSTED BLUE,5,4.1,16499,24999,34,Available,17978,...,128.0,0.0,1,1,0,0,1,1,0,120810362.2
2753,MOTOROLA,MOTOROLA G62,MIDNIGHT GRAY,5,4.1,16499,24999,34,Available,17978,...,128.0,0.0,1,1,0,0,1,1,0,120810362.2
50,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16300,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
30,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,16385,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
16,SAMSUNG,SAMSUNG GALAXY M33,MYSTIQUE GREEN,5,4.2,16490,24999,34,Available,11095,...,128.0,0.0,0,1,0,1,1,0,0,37884987.0
1313,realme,REALME 9 SE,STARRY GLOW,5,4.3,17999,26999,33,Currently unavailable,4758,...,128.0,1024.0,1,1,0,1,1,1,0,7999625.4
1321,realme,REALME 9 SE,STARRY GLOW,5,4.3,17999,26999,33,Currently unavailable,4758,...,128.0,1024.0,1,1,0,1,1,1,0,7672275.0
1090,realme,REALME 9 SE,AZURE GLOW,5,4.3,17999,26999,33,Available,4758,...,128.0,1024.0,1,1,0,1,1,1,0,7672275.0
39,SAMSUNG,SAMSUNG GALAXY M33,DEEP OCEAN BLUE,5,4.2,17379,25999,33,Available,3173,...,128.0,0.0,0,1,0,1,1,0,0,2878545.6
344,SAMSUNG,SAMSUNG GALAXY M51,CELESTIAL BLACK,0,4.4,17990,26999,33,Currently unavailable,2021,...,128.0,0.0,0,1,0,1,1,0,0,1307182.8
